# UNESCO Heritage Sites Risk Modeling - Risk Analysis

## Notebook 02: Risk Score Analysis and Insights

**Purpose**: Analyze calculated risk scores, identify patterns, and generate insights about heritage site vulnerabilities.

**Contents**:
1. Load risk scores and merge with site data
2. Risk score distributions and statistics
3. Sub-score analysis (urban, climate, seismic, fire, flood, coastal)
4. Correlation analysis between risk factors
5. High-risk site identification
6. Anomaly detection results
7. Geographic risk patterns

In [ ]:
# Import required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import text
import warnings

# Import project modules
import sys
sys.path.append('..')
from src.db.connection import get_session, engine
from config.settings import DEFAULT_WEIGHTS

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## 1. Load Risk Scores and Site Data

In [ ]:
# Load sites with risk scores
query = """
SELECT 
    hs.id, hs.whc_id, hs.name, hs.country, hs.category,
    hs.in_danger, hs.area_hectares,
    ST_X(hs.geom) as longitude, ST_Y(hs.geom) as latitude,
    rs.urban_density_score,
    rs.climate_anomaly_score,
    rs.seismic_risk_score,
    rs.fire_risk_score,
    rs.flood_risk_score,
    rs.coastal_risk_score,
    rs.composite_risk_score,
    rs.risk_level,
    rs.is_anomaly,
    rs.isolation_forest_score
FROM unesco_risk.heritage_sites hs
JOIN unesco_risk.risk_scores rs ON hs.id = rs.site_id
ORDER BY rs.composite_risk_score DESC;
"""

risk_df = pd.read_sql(query, engine)
print(f"Loaded {len(risk_df)} sites with risk scores")
risk_df.head()

In [ ]:
# Display risk weights being used
print("=" * 60)
print("RISK WEIGHTS")
print("=" * 60)
for factor, weight in DEFAULT_WEIGHTS.items():
    print(f"{factor:30s}: {weight:.2f} ({weight*100:.0f}%)")
print(f"\nTotal: {sum(DEFAULT_WEIGHTS.values()):.2f}")

## 2. Risk Score Distributions

In [ ]:
# Risk level distribution
print("=" * 60)
print("RISK LEVEL DISTRIBUTION")
print("=" * 60)
risk_level_counts = risk_df['risk_level'].value_counts()
print(risk_level_counts)
print(f"\nPercentages:")
print((risk_level_counts / len(risk_df) * 100).round(1))

In [ ]:
# Risk level bar chart
plt.figure(figsize=(10, 6))
risk_order = ['low', 'medium', 'high', 'critical']
colors_map = {'low': 'green', 'medium': 'yellow', 'high': 'orange', 'critical': 'red'}

counts = risk_df['risk_level'].value_counts().reindex(risk_order, fill_value=0)
bars = plt.bar(range(len(counts)), counts.values, 
               color=[colors_map[level] for level in counts.index])

plt.xticks(range(len(counts)), counts.index.str.capitalize())
plt.ylabel('Number of Sites', fontsize=12)
plt.title('Risk Level Distribution', fontsize=14, fontweight='bold')
plt.grid(True, axis='y', alpha=0.3)

# Add count labels on bars
for i, (bar, count) in enumerate(zip(bars, counts.values)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
             str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Composite risk score histogram
plt.figure(figsize=(12, 6))
plt.hist(risk_df['composite_risk_score'], bins=50, edgecolor='black', alpha=0.7)
plt.axvline(risk_df['composite_risk_score'].mean(), color='red', 
            linestyle='--', linewidth=2, label=f'Mean: {risk_df["composite_risk_score"].mean():.3f}')
plt.axvline(risk_df['composite_risk_score'].median(), color='blue', 
            linestyle='--', linewidth=2, label=f'Median: {risk_df["composite_risk_score"].median():.3f}')
plt.xlabel('Composite Risk Score', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Composite Risk Scores', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Sub-Score Analysis

In [ ]:
# Statistics for all sub-scores
score_columns = [
    'urban_density_score',
    'climate_anomaly_score',
    'seismic_risk_score',
    'fire_risk_score',
    'flood_risk_score',
    'coastal_risk_score'
]

print("=" * 60)
print("SUB-SCORE STATISTICS")
print("=" * 60)
print(risk_df[score_columns].describe().T.round(3))

In [ ]:
# Box plots for all sub-scores
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

score_labels = [
    'Urban Density',
    'Climate Anomaly',
    'Seismic Risk',
    'Fire Risk',
    'Flood Risk',
    'Coastal Risk'
]

for i, (col, label) in enumerate(zip(score_columns, score_labels)):
    axes[i].boxplot(risk_df[col].dropna(), vert=True)
    axes[i].set_title(label, fontsize=12, fontweight='bold')
    axes[i].set_ylabel('Score', fontsize=10)
    axes[i].grid(True, alpha=0.3)
    axes[i].set_ylim(-0.05, 1.05)

plt.suptitle('Risk Sub-Score Distributions', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Correlation Analysis

In [ ]:
# Correlation matrix
corr_matrix = risk_df[score_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1,
            xticklabels=[s.replace('_score', '').replace('_', ' ').title() for s in score_columns],
            yticklabels=[s.replace('_score', '').replace('_', ' ').title() for s in score_columns])
plt.title('Risk Factor Correlation Matrix', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nStrong correlations (|r| > 0.5):")
for i in range(len(corr_matrix)):
    for j in range(i+1, len(corr_matrix)):
        if abs(corr_matrix.iloc[i, j]) > 0.5:
            print(f"{score_columns[i]:25s} <-> {score_columns[j]:25s}: {corr_matrix.iloc[i, j]:6.3f}")

## 5. High-Risk Site Identification

In [ ]:
# Top 20 highest risk sites
print("=" * 80)
print("TOP 20 HIGHEST RISK SITES")
print("=" * 80)

top_20 = risk_df.nlargest(20, 'composite_risk_score')[[
    'name', 'country', 'category', 'composite_risk_score', 'risk_level',
    'urban_density_score', 'seismic_risk_score', 'climate_anomaly_score'
]]

print(top_20.to_string(index=False))

In [ ]:
# Sites in danger status vs risk level
danger_risk = pd.crosstab(risk_df['in_danger'], risk_df['risk_level'])

print("\n" + "=" * 60)
print("IN DANGER STATUS vs RISK LEVEL")
print("=" * 60)
print(danger_risk)

# Are sites officially "in danger" correlated with high risk scores?
in_danger_avg_risk = risk_df.groupby('in_danger')['composite_risk_score'].mean()
print(f"\nAverage risk score - Not in danger: {in_danger_avg_risk[False]:.3f}")
if True in in_danger_avg_risk:
    print(f"Average risk score - In danger:     {in_danger_avg_risk[True]:.3f}")

## 6. Anomaly Detection Results

In [ ]:
# Anomaly statistics
anomaly_count = risk_df['is_anomaly'].sum()
anomaly_pct = (anomaly_count / len(risk_df)) * 100

print("=" * 60)
print("ANOMALY DETECTION RESULTS")
print("=" * 60)
print(f"Total sites: {len(risk_df)}")
print(f"Anomalous sites: {anomaly_count} ({anomaly_pct:.1f}%)")
print(f"Normal sites: {len(risk_df) - anomaly_count} ({100-anomaly_pct:.1f}%)")

In [ ]:
# Anomalous sites list
if anomaly_count > 0:
    print("\nAnomalous Sites (sorted by Isolation Forest score):")
    anomalies = risk_df[risk_df['is_anomaly'] == True][[
        'name', 'country', 'composite_risk_score', 'isolation_forest_score',
        'urban_density_score', 'seismic_risk_score', 'climate_anomaly_score'
    ]].sort_values('isolation_forest_score').head(20)
    
    print(anomalies.to_string(index=False))

In [ ]:
# Isolation Forest score distribution
if 'isolation_forest_score' in risk_df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    axes[0].hist(risk_df['isolation_forest_score'], bins=50, edgecolor='black', alpha=0.7)
    axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='Decision Boundary')
    axes[0].set_xlabel('Isolation Forest Score', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('Isolation Forest Score Distribution', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Scatter: IF score vs composite risk
    colors = risk_df['is_anomaly'].map({True: 'red', False: 'blue'})
    axes[1].scatter(risk_df['composite_risk_score'], risk_df['isolation_forest_score'],
                   c=colors, alpha=0.5, s=30)
    axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
    axes[1].set_xlabel('Composite Risk Score', fontsize=12)
    axes[1].set_ylabel('Isolation Forest Score', fontsize=12)
    axes[1].set_title('Anomaly Score vs Risk Score', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8, label='Anomaly'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=8, label='Normal')
    ]
    axes[1].legend(handles=legend_elements)
    
    plt.tight_layout()
    plt.show()

## 7. Geographic Risk Patterns

In [ ]:
# Average risk by country (top 15)
country_risk = risk_df.groupby('country').agg({
    'composite_risk_score': ['mean', 'count']
}).round(3)
country_risk.columns = ['avg_risk', 'site_count']
country_risk = country_risk[country_risk['site_count'] >= 3]  # At least 3 sites
country_risk = country_risk.sort_values('avg_risk', ascending=False).head(15)

print("=" * 60)
print("TOP 15 COUNTRIES BY AVERAGE RISK")
print("(Countries with at least 3 sites)")
print("=" * 60)
print(country_risk)

In [ ]:
# Geographic scatter plot colored by risk
fig, ax = plt.subplots(figsize=(16, 10))

scatter = ax.scatter(risk_df['longitude'], risk_df['latitude'],
                    c=risk_df['composite_risk_score'],
                    cmap='RdYlGn_r',  # Red-Yellow-Green reversed
                    s=50, alpha=0.6, edgecolors='black', linewidth=0.5)

# Mark anomalies with black circles
if anomaly_count > 0:
    anomalies = risk_df[risk_df['is_anomaly'] == True]
    ax.scatter(anomalies['longitude'], anomalies['latitude'],
              facecolors='none', edgecolors='black', s=200, linewidth=2,
              label='Anomaly', zorder=10)

plt.colorbar(scatter, ax=ax, label='Composite Risk Score')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.set_title('Geographic Distribution of Risk Scores', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

## Summary and Key Insights

This notebook analyzed the risk scores for UNESCO heritage sites. Key findings:

1. **Risk Distribution**: [View the actual distribution from your run]
2. **Top Risk Factors**: [Identify which sub-scores have highest values]
3. **High-Risk Sites**: [Note the top sites from your analysis]
4. **Anomalies**: [Describe anomalous patterns detected]
5. **Geographic Patterns**: [Note any regional risk patterns]

**Next Steps**:
- Investigate specific high-risk sites in detail
- Explore mitigation strategies for different risk types
- Create interactive visualizations (Notebook 03)